## Setup


In this first cell we''ll load the necessary libraries and setup some logging and display options.

In [1]:
import math

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

%matplotlib inline

Define a function that displays the contents of a NetCDF.

In [2]:
def display_netcdf(nc_path):
    
    # Open NetCDF as xarray DataSet and print the info attribute.
    print(xr.open_dataset(nc_path).info)

Define a function that creates a copy of a NetCDF. 

In [3]:
def netcdf_copy(path_orig,
                path_copy):
    
    # Open the NetCDF as an xarray DataSet and write it back out as NetCDF.
    xr.open_dataset(path_orig, decode_times=False).to_netcdf(path=path_copy)

Copy the labels NetCDF into a corresponding predicted labels NetCDF. We expect to update the label variable's values in this NetCDF with values predicted by an AI model that we'll develop and train below.

In [4]:
# paths to the computed and predicted labels NetCDF files
path_labels = 'C:/home/cam_learn/fv091x180L26_dry_HS.cam.h1.2000-12-27-00000_lowres.nc'
path_predictions = 'C:/home/cam_learn/fv091x180L26_dry_HS.cam.h2.2000-12-27-00000_lowres.nc'

# make a copy of the computed labels NetCDF as the label predictions NetCDF
netcdf_copy(path_labels, path_predictions)
display_netcdf(path_predictions)

<bound method Dataset.info of <xarray.Dataset>
Dimensions:       (ilev: 27, lat: 12, lev: 26, lon: 23, nbnd: 2, slat: 90, slon: 180, time: 720)
Coordinates:
  * ilev          (ilev) float64 2.194 4.895 9.882 18.05 29.84 44.62 61.61 ...
  * lat           (lat) float64 -90.0 -74.0 -58.0 -42.0 -26.0 -10.0 6.0 22.0 ...
  * lev           (lev) float64 3.545 7.389 13.97 23.94 37.23 53.11 70.06 ...
  * lon           (lon) float64 0.0 16.0 32.0 48.0 64.0 80.0 96.0 112.0 ...
  * slat          (slat) float64 -89.0 -87.0 -85.0 -83.0 -81.0 -79.0 -77.0 ...
  * slon          (slon) float64 -1.0 1.0 3.0 5.0 7.0 9.0 11.0 13.0 15.0 ...
  * time          (time) datetime64[ns] 2000-12-27 2000-12-27T00:30:00 ...
Dimensions without coordinates: nbnd
Data variables:
    P0            float64 ...
    PTTEND        (time, lev, lat, lon) float32 ...
    PUTEND        (time, lev, lat, lon) float32 ...
    PVTEND        (time, lev, lat, lon) float32 ...
    ch4vmr        (time) float64 ...
    co2vmr        (tim

Next we'll load our flow variables (features) and time tendency forcings (labels) datasets into Xarray Dataset objects.

In [5]:
path_features = 'C:/home/cam_learn/fv091x180L26_dry_HS.cam.h0.2000-12-27-00000_lowres.nc'
ds_features = xr.open_dataset(path_features, decode_times=False)
ds_labels = xr.open_dataset(path_labels, decode_times=False)

In [6]:
ds_features.info

<bound method Dataset.info of <xarray.Dataset>
Dimensions:       (ilev: 27, lat: 12, lev: 26, lon: 23, nbnd: 2, slat: 90, slon: 180, time: 720)
Coordinates:
  * ilev          (ilev) float64 2.194 4.895 9.882 18.05 29.84 44.62 61.61 ...
  * lat           (lat) float64 -90.0 -74.0 -58.0 -42.0 -26.0 -10.0 6.0 22.0 ...
  * lev           (lev) float64 3.545 7.389 13.97 23.94 37.23 53.11 70.06 ...
  * lon           (lon) float64 0.0 16.0 32.0 48.0 64.0 80.0 96.0 112.0 ...
  * slat          (slat) float64 -89.0 -87.0 -85.0 -83.0 -81.0 -79.0 -77.0 ...
  * slon          (slon) float64 -1.0 1.0 3.0 5.0 7.0 9.0 11.0 13.0 15.0 ...
  * time          (time) float64 0.0 0.02083 0.04167 0.0625 0.08333 0.1042 ...
Dimensions without coordinates: nbnd
Data variables:
    P0            float64 ...
    PS            (time, lat, lon) float32 ...
    T             (time, lev, lat, lon) float32 ...
    U             (time, lev, lat, lon) float32 ...
    V             (time, lev, lat, lon) float32 ...
    ch4v

In [7]:
ds_labels.info

<bound method Dataset.info of <xarray.Dataset>
Dimensions:       (ilev: 27, lat: 12, lev: 26, lon: 23, nbnd: 2, slat: 90, slon: 180, time: 720)
Coordinates:
  * ilev          (ilev) float64 2.194 4.895 9.882 18.05 29.84 44.62 61.61 ...
  * lat           (lat) float64 -90.0 -74.0 -58.0 -42.0 -26.0 -10.0 6.0 22.0 ...
  * lev           (lev) float64 3.545 7.389 13.97 23.94 37.23 53.11 70.06 ...
  * lon           (lon) float64 0.0 16.0 32.0 48.0 64.0 80.0 96.0 112.0 ...
  * slat          (slat) float64 -89.0 -87.0 -85.0 -83.0 -81.0 -79.0 -77.0 ...
  * slon          (slon) float64 -1.0 1.0 3.0 5.0 7.0 9.0 11.0 13.0 15.0 ...
  * time          (time) float64 0.0 0.02083 0.04167 0.0625 0.08333 0.1042 ...
Dimensions without coordinates: nbnd
Data variables:
    P0            float64 ...
    PTTEND        (time, lev, lat, lon) float32 ...
    PUTEND        (time, lev, lat, lon) float32 ...
    PVTEND        (time, lev, lat, lon) float32 ...
    ch4vmr        (time) float64 ...
    co2vmr        

Look at the time variable in order to work out the initial date, number of steps, units, etc.

In [14]:
ds_features.variables['time']

<xarray.IndexVariable 'time' (time: 720)>
array([ 0.      ,  0.020833,  0.041667, ..., 14.9375  , 14.958333, 14.979167])
Attributes:
    long_name:  time
    units:      days since 2000-12-27 00:00:00
    calendar:   noleap
    bounds:     time_bnds

Make sure we have the same time values for the targets data.

In [15]:
if (ds_features.variables['time'].values != ds_labels.variables['time'].values).any():
    print('ERROR: Non-matching time values')
else:
    print("OK: time values match as expected")

OK: time values match as expected


#### Define a function that creates a Series of timestamp values corresponding to the NetCDF's time values.

In [16]:
from datetime import datetime, timedelta
def extract_timestamps(ds,
                       year,
                       month,
                       day):
    
    # Cook up an initial datetime object based on our specified initial date.
    initial = datetime(2000, 12, 27)
    
    # Create an array of datetime objects from the time values (assumed to be in units of days since the inital date).
    times = ds.variables['time'].values
    datetimes = np.empty(shape=times.shape, dtype='datetime64[m]')
    for i in range(datetimes.size):
        datetimes[i] = initial + timedelta(days=times[i])

    # Put the array into a Series and return it.    
    return pd.Series(datetimes)

#### Create a Series of datetime values from the DataSet's (NetCDF's) time coordinate variable's values.
These will be used as time indices corresponding to feature and label values.

In [17]:
timestamps = extract_timestamps(ds_features, 2000, 12, 27)
timestamps.head()

0   2000-12-27 00:00:00
1   2000-12-27 00:30:00
2   2000-12-27 01:00:00
3   2000-12-27 01:30:00
4   2000-12-27 02:00:00
dtype: datetime64[ns]

## Feature and target selection

As features we'll use the following flow variables:

* U (west-east (zonal) wind, m/s)
* V (south-north (meridional) wind, m/s)
* T (temperature, K)
* PS (surface pressure, Pa)

Time tendency forcings are the targets (labels) that our model should learn to predict.

* PTTEND (time tendency forcing value corresponding to the temperature variable)
* PUTEND (time tendency forcing value corresponding to zonal wind)
* PVTEND (time tendency forcing value corresponding to meridional wind)

Eventually we'll train/fit our model for an entire global 3-D grid, but for this example we'll select all lat/lon/time combinations for a single level (elevation).

In [20]:
ps = pd.Series(ds_features.variables['PS'].values[:, :, :].flatten())
t = pd.Series(ds_features.variables['T'].values[:, 0, :, :].flatten())
u = pd.Series(ds_features.variables['U'].values[:, 0, :, :].flatten())
v = pd.Series(ds_features.variables['V'].values[:, 0, :, :].flatten())
pttend = pd.Series(ds_labels.variables['PTTEND'].values[:, 0, :, :].flatten())
putend = pd.Series(ds_labels.variables['PUTEND'].values[:, 0, :, :].flatten())
pvtend = pd.Series(ds_labels.variables['PVTEND'].values[:, 0, :, :].flatten())

Convert to Pandas DataFrames containing inputs (features) and outputs (label/target) for use when predicting time tendency forcings.

In [21]:
df_features = pd.DataFrame({'timestamp': timestamps,
                            'PS': ps,
                            'T': t,
                            'U': u,
                            'V': v})
df_features.set_index('timestamp', inplace=True)
df_features.head()

,PS,T,U,V
timestamp,,,,
2000-12-27 00:00:00,101099.0625,210.862564,-0.814972,-0.280670
2000-12-27 00:30:00,101099.0625,210.862564,-0.706038,-0.494434
2000-12-27 01:00:00,101099.0625,210.862564,-0.542403,-0.669891
2000-12-27 01:30:00,101099.0625,210.862564,-0.336744,-0.793447
2000-12-27 02:00:00,101099.0625,210.862564,-0.104996,-0.855530


In [22]:
df_targets = pd.DataFrame({'timestamp': timestamps,
                           'PTTEND': pttend,
                           'PUTEND': putend,
                           'PVTEND': pvtend})
df_targets.set_index('timestamp', inplace=True)
df_targets.head()

,PTTEND,PUTEND,PVTEND
timestamp,,,
2000-12-27 00:00:00,-0.000003,0.0,0.0
2000-12-27 00:30:00,-0.000003,0.0,0.0
2000-12-27 01:00:00,-0.000003,0.0,0.0
2000-12-27 01:30:00,-0.000003,0.0,0.0
2000-12-27 02:00:00,-0.000003,0.0,0.0


## Split the data into training and testing datasets

We'll use an initial split of 75% for training and 25% for testing.

In [25]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df_features, df_targets, test_size=0.25, random_state=4)

## Create the linear regression model

In [26]:
from sklearn import linear_model
model = linear_model.LinearRegression()

## Train and evaluate the model

Train the model by fitting to the training dataset. Then predict the labels using the test features, and get the RMSE compared against the test labels.

In [43]:
# fit the model
history = model.fit(x_train, y_train)

# root mean square error 
rmse = np.sqrt(np.mean((model.predict(x_test) - y_test)**2))
print("RMSE:\n{}".format(rmse))

model.

RMSE:
PTTEND    1.959330e-12
PUTEND    0.000000e+00
PVTEND    0.000000e+00
dtype: float32


## Build a corresponding predictions dataset

Next we'll loop over each level of the features/labels datasets and use each of these in turn to train the model, which we'll then use to compute/predict corresponding labels. These predictions will be written into the predictions NetCDF (TODO along with the RMSE vs. original labels as separate/corresponding variables).

#### Define a function to extract a feature dataset for a specific level incorporating the feature variables (PS, T, U, and V).

(Incorporating timestamps in case they're useful later when utilizing models that are relevant to timeseries, but in this first use case, i.e. with a simple linear regression model, this appears to be inconsequential/superfluous.)

In [55]:
def extract_features(ds_features,
                     level_index,
                     timestamps):
    
    # Create a DataFrame from the feature variables PS, T, U, and V.
    ps = pd.Series(ds_features.variables['PS'].values[:, :, :].flatten())
    t = pd.Series(ds_features.variables['T'].values[:, level_index, :, :].flatten())
    u = pd.Series(ds_features.variables['U'].values[:, level_index, :, :].flatten())
    v = pd.Series(ds_features.variables['V'].values[:, level_index, :, :].flatten())
    df_features = pd.DataFrame({'timestamp': timestamps,
                                'PS': ps,
                                'T': t,
                                'U': u,
                                'V': v})
    df_features.set_index('timestamp', inplace=True)

    return df_features

#### Define a function to extract training and test datasets for a specific level and single target variable, using all feature variables (PS, T, U, and V).

(Incorporating timestamps in case they're useful later when utilizing models that are relevant to timeseries, but in this first use case, i.e. with a simple linear regression model, this appears to be inconsequential/superfluous.)

In [56]:
def extract_train_test(ds_features,
                       ds_labels,
                       timestamps,
                       level_index,
                       label,
                       test_percentage):
    
    # Create a DataFrame from the feature variables PS, T, U, and V.
    df_features = extract_features(ds_features, level_index, timestamps)

    # Create a labels DataFrame from the specified label variable.
    target = pd.Series(ds_labels.variables[label].values[:, level_index, :, :].flatten())
    df_target = pd.DataFrame({'timestamp': timestamps,
                               label: target})
    df_target.set_index('timestamp', inplace=True)

    # Pull training and test datasets from the features and target DataFrames.
    x_train, x_test, y_train, y_test = train_test_split(df_features, 
                                                        df_target, 
                                                        test_size=test_percentage,
                                                        random_state=4)

    return x_train, x_test, y_train, y_test

#### Determine original shape of a single level

In [71]:
# Get the original shape of the variables (assumed to be the same for features and labels), 
# and use this to establish the shape we'll use for reshaping our predictions, which will 
# initally come out of the model as a flattened array.
shape = ds_labels.variables['PVTEND'].values.shape
shape_single_level = (shape[0], shape[2], shape[3])

### Train the model per level/label and predict values, writing the predictions to NetCDF.

In [75]:
# Open the predictions NetCDF as an xarray DataSet.
ds_predictions = xr.open_dataset(path_predictions, lock=False)

# Loop over levels, using a level index
for lev_ix in range(ds_features.dims['lev']):
    
    # Loop over each label, in order to predict each in isolation
    # TODO is this necessary/optimal, or can we just as well do all three simultaneously? TEST THIS
    # i.e. is univariate any more accurate than multivariate when using sklearn's LRM?
    for forcing_label in ['PTTEND', 'PUTEND', 'PVTEND']:
    
        # Extract training and testing datasets.
        x_train, x_test, y_train, y_test = extract_train_test(ds_features,
                                                              ds_labels,
                                                              timestamps,
                                                              lev_ix,
                                                              forcing_label,
                                                              0.25)
        
        # Train the model, then predict the relevant forcing tendencies using the trained model.
        model.fit(x_train, y_train)
        input_features = extract_features(ds_features, lev_ix, timestamps)
        predictions = model.predict(input_features)
        
        # Write the prediction values into the NetCDF at the relevant level.
        values = np.reshape(predictions, shape_single_level)
        ds_predictions.variables[forcing_label].values[:, lev_ix, :, :] = values
        
# Write the predictions dataset back as a NetCDF, overwriting the previous copy.
ds_predictions.to_netcdf('C:\\home\\cam_learn\\fv091x180L26_dry_HS.cam.PREDICTED.2000-12-27-00000_lowres.nc', mode='w')

### Comparison with computed results
At this point we can compare the predictions against the computed labels to determine how accurately the linear regression model matched the results of the climate model. Once we're satisfied that the errors are marginal then we can use the LRM to predict labels using new inputs for the same features, and then test to se how closely the predictions match to the known results for those inputs. The goals are to develop an appropriate AI model (not ncessarily a linear regression model as is used above) and to determine the optimal parameters of the AI model where it can replace (and hopefully improve upon the results) of the existing climate model.  